#SoMe Topic Modeling Notebook | Release canvas 1 📖


In [1]:
# Installations
import sys
if 'google.colab' in sys.modules:
    !pip install emoji --upgrade
    !pip install pandas-profiling==2.*
    !pip install plotly==4.*
    !python -m spacy download en_core_web_lg
    !pip install pyldavis

     |████████████████████████████████| 51kB 1.7MB/s 
  Created wheel for emoji: filename=emoji-0.5.4-cp36-none-any.whl size=42176 sha256=636c021ff4e469866a19547f051bddd1ecf259d9cf73905d5665bef35dd1907c
  Stored in directory: /root/.cache/pip/wheels/2a/a9/0a/4f8e8cce8074232aba240caca3fade315bb49fac68808d1a9c
Successfully built emoji
     |████████████████████████████████| 266kB 2.7MB/s 
     |████████████████████████████████| 61kB 4.3MB/s 
     |████████████████████████████████| 614kB 6.5MB/s 
     |████████████████████████████████| 3.1MB 14.3MB/s 
     |████████████████████████████████| 71kB 7.2MB/s 
     |████████████████████████████████| 296kB 29.7MB/s 
  Created wheel for htmlmin: filename=htmlmin-0.1.12-cp36-none-any.whl size=27084 sha256=9fb36be39627fb21d560c5039432223ab8f0db723b8616fb71a948f07fce6156
  Stored in directory: /root/.cache/pip/wheels/43/07/ac/7c5a9d708d65247ac1f94066cf1db075540b85716c30255459
  Created wheel for confuse: filename=confuse-1.1.0-cp36-none-any.whl size

     |████████████████████████████████| 827.9MB 1.1MB/s 
  Created wheel for en-core-web-lg: filename=en_core_web_lg-2.2.5-cp36-none-any.whl size=829180944 sha256=943b4d85ac2f302b99b816911a9f5d06a6908857920e4f505f484d47f5ded5b0
  Stored in directory: /tmp/pip-ephem-wheel-cache-fb3la0po/wheels/2a/c1/a6/fc7a877b1efca9bc6a089d6f506f16d3868408f9ff89f8dbfc
Successfully built en-core-web-lg
✔ Download and installation successful
You can now load the model via spacy.load('en_core_web_lg')
     |████████████████████████████████| 1.6MB 2.7MB/s 
     |████████████████████████████████| 552kB 13.1MB/s 
  Created wheel for pyldavis: filename=pyLDAvis-2.1.2-py2.py3-none-any.whl size=97711 sha256=1fe64102312919d5356bb7f647e9166e455b6ea9f1d15096d61fab4b4ab56eba
  Stored in directory: /root/.cache/pip/wheels/98/71/24/513a99e58bb6b8465bae4d2d5e9dba8f0bef8179e3051ac414
  Created wheel for funcy: filename=funcy-1.14-py2.py3-none-any.whl size=32042 sha256=0f9b7e3c48c39f982a1ee8ef06034cc2b4a339437cb4480c58f

In [0]:
# Required Libraries

#Base and Cleaning 
import json
import requests
import pandas as pd
import numpy as np
import emoji
import regex
import re

#Visualizations
import plotly.express as px
import seaborn as sns
import matplotlib as plt 
import pyLDAvis.gensim

#Natural Language Processing (NLP)
import spacy
from spacy.tokenizer import Tokenizer
from gensim.corpora import Dictionary
from gensim.models.ldamulticore import LdaMulticore
from gensim.models.coherencemodel import CoherenceModel

##Data Cleaning 🧹

In [0]:
# Loading the JSON file 
url_elon = 'https://raw.githubusercontent.com/Lambda-School-Labs/social-media-strategy-ds/feature/topic/python_notebooks/elonmusk_followers_english.json'
url_dutchbros = 'https://raw.githubusercontent.com/Lambda-School-Labs/social-media-strategy-ds/feature/topic/python_notebooks/dutchbros_followers.json'
df = requests.get(url_dutchbros).json()
df

In [0]:
# Converting the dataset to pandas DataFrame and renaming the columns 
df = pd.DataFrame(df.values())
df = df.rename(columns={0:'tweets'})
df.head()

,tweets
0,This kid will forever be a legend 😂 https://t....
1,"If you truly believe Lebrons mindset, competit..."
2,BUTTLICKER! OUR PRICES HAVE NEVER BEEN LOWER!!!
3,@Bhuvan_Bam ❤️❤️
4,I'm not crying you're crying.\nhttps://t.co/Bc...


In [0]:
#Removing emojies from text and and puttin them in a new column
#Refrence 1 : https://stackoverflow.com/questions/43146528/how-to-extract-all-the-emojis-from-text
#Refrence 2 : https://stackoverflow.com/questions/33404752/removing-emojis-from-a-string-in-python
#Refrence 3 : https://stackoverflow.com/questions/11331982/how-to-remove-any-url-within-a-string-in-python

### Question: How to combine all functions into one 
def extract_emoji(text):
    '''
    Extracts emojies from text
    '''
    emoji_list = []
    data = regex.findall(r'\X', text)
    for word in data:
        if any(char in emoji.UNICODE_EMOJI for char in word):
            emoji_list.append(word)
    return emoji_list

def emoji_free_text(text):
    '''
    Cleans text from emojies
    '''
    emoji_list_1 = [c for c in text if c in emoji.UNICODE_EMOJI]
    clean_text = ' '.join([str for str in text.split() if not any(i in str for i in emoji_list_1)])
    return clean_text

def url_free_text(text):
    '''
    Cleans text from urls
    '''
    text = re.sub(r'http\S+', '', text)
    return text

#Create a new column with emojies from each tweet
df['tweet_emojies'] = df['tweets'].apply(lambda x : extract_emoji(x))

#Create a new column with cleaned tweets from emojies
df['emoji_free_tweets'] = df['tweets'].apply(lambda x : emoji_free_text(x))

#Create a new column with url free tweets
df['url_free_tweets'] = df['emoji_free_tweets'].apply(lambda x : url_free_text(x))

df.head()

,tweets,tweet_emojies,emoji_free_tweets,url_free_tweets
0,This kid will forever be a legend 😂 https://t....,[😂],This kid will forever be a legend https://t.co...,This kid will forever be a legend
1,"If you truly believe Lebrons mindset, competit...",[],"If you truly believe Lebrons mindset, competit...","If you truly believe Lebrons mindset, competit..."
2,BUTTLICKER! OUR PRICES HAVE NEVER BEEN LOWER!!!,[],BUTTLICKER! OUR PRICES HAVE NEVER BEEN LOWER!!!,BUTTLICKER! OUR PRICES HAVE NEVER BEEN LOWER!!!
3,@Bhuvan_Bam ❤️❤️,"[❤️, ❤️]",@Bhuvan_Bam,@Bhuvan_Bam
4,I'm not crying you're crying.\nhttps://t.co/Bc...,[],I'm not crying you're crying. https://t.co/BcF...,I'm not crying you're crying.


##Topic Modeling ㊙️

###Tokenizing 🕵🏻‍♂️

In [0]:
# Load spacy
# Make sure to restart the runtime after running installations and libraries tab
nlp = spacy.load('en_core_web_lg')

In [0]:
# Get Tokens 
tokens = []

for doc in nlp.pipe(df['url_free_tweets'], batch_size=500):
    
    doc_tokens = []
    
    for token in doc:
        if (token.is_stop == False) & (token.is_punct == False):
            doc_tokens.append(token.lemma_.lower())
            
    tokens.append(doc_tokens)
        
df['tokens'] = tokens

del tokens

In [0]:
df['tokens'].head()

0                               [kid, forever, legend]
1    [truly, believe, lebrons, mindset, competitive...
2                             [buttlicker, price, low]
3                                        [@bhuvan_bam]
4                                           [cry, cry]
Name: tokens, dtype: object

###id2word 📒

In [0]:
# Create a id2word dictionary
id2word = Dictionary(df['tokens'])
print(len(id2word))

22210


In [0]:
# Filtering Extremes
id2word.filter_extremes(no_below=2, no_above=.99)
print(len(id2word))

8310


###Corpus Object & Getting Topics 📚

In [0]:
# Creating a corpus object 
corpus = [id2word.doc2bow(d) for d in df['tokens']]

In [0]:
# Instantiating a LDA model 
model = LdaMulticore(corpus=corpus, num_topics=10, id2word=id2word, workers=12, passes=5)

In [0]:
# Filtering for words 
words = [re.findall(r'"([^"]*)"',t[1]) for t in model.print_topics()]

In [0]:
# Create Topics
topics = [' '.join(t[0:5]) for t in words]

In [0]:
# Getting the topics
for id, t in enumerate(topics): 
    print(f"------ Topic {id} ------")
    print(t, end="\n\n")

------ Topic 0 ------
people like government new mask

------ Topic 1 ------
obamagate time amp need right

------ Topic 2 ------
$ amp know btc bitcoin

------ Topic 3 ------
time like year good think

------ Topic 4 ------
amp like follow   retweet

------ Topic 5 ------
day   amp 2 new

------ Topic 6 ------
work time   go state

------ Topic 7 ------
  think new people good

------ Topic 8 ------
people work go want  

------ Topic 9 ------
trump day say love like



### Topic Distance Visualization 📈

*   List item
*   List item



In [0]:
#Creating Topic Distance Visualization 
pyLDAvis.enable_notebook()
pyLDAvis.gensim.prepare(model, corpus, id2word)

PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
2     -0.063187  0.030633       1        1  12.766200
4     -0.046396  0.021477       2        1  12.598732
3      0.045059 -0.020323       3        1  11.061243
9      0.006408 -0.059356       4        1  10.912730
6     -0.005587 -0.067669       5        1   9.708640
7      0.042492 -0.009301       6        1   9.173834
5     -0.124710  0.045252       7        1   8.703091
0      0.020639 -0.057697       8        1   8.635661
8      0.034109 -0.007249       9        1   8.507177
1      0.091174  0.124232      10        1   7.932688, topic_info=           Term        Freq       Total Category  logprob  loglift
1203          $  605.000000  605.000000  Default  30.0000  30.0000
804   obamagate  184.000000  184.000000  Default  29.0000  29.0000
3149        btc  144.000000  144.000000  Default  28.0000  28.0000
365      people  613.000000  613.000000  Default  27.0000  27.0000
807       trump  324.000000  324.000000  Default  26.0000  26.0000
...         ...         ...         ...      ...      ...      ...
365      people   26.649826  613.343567  Topic10  -5.6721  -0.6020
254        want   23.935995  345.415863  Topic10  -5.7795  -0.1352
94        thank   22.118052  213.582352  Topic10  -5.8585   0.2665
9          life   21.591526  181.491318  Topic10  -5.8826   0.4053
439       great   21.626474  193.116013  Topic10  -5.8810   0.3448

[835 rows x 6 columns], token_table=      Topic      Freq Term
term                      
141       1  0.116361     
141       2  0.149834     
141       3  0.052601     
141       4  0.054195     
141       5  0.094045     
...     ...       ...  ...
7476     10  0.815661   وہ
7488     10  0.932699  ہیں
4576      2  0.079441    ★
4576      5  0.079441    ★
4576      7  0.794410    ★

[2758 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[3, 5, 4, 10, 7, 8, 6, 1, 9, 2])

In [0]:
# Steps to Score Training Documents
# Already have BOW Represented called 'corpus'

distro = [lda[d] for d in corpus]
num_topics = 20

def update(doc):
        d_dist = {k:0 for k in range(0,num_topics)}
        for topic in doc:
            # Topic is Tuple where the first part is the topic id
            # the second part is the topic distribution in that doc
            d_dist[topic[0]] = topic[1]
        return d_dist
    
new_distro = [update(d) for d in distro]

NameError: ignored